# 1.问题分析

## 1.1.问题类别

## 1.2.特征分析 

## 1.3.数据挖掘

# 2.准备工作

## 2.1.准备工具包

In [1]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math


from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import StratifiedKFold


%matplotlib inline

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 2.2.原始数据准备(不进行任何清洗)

In [2]:
train_raw = pd.read_csv('../data/public_raw.train.csv')
test_raw = pd.read_csv('../data/public_raw.test.csv')

train_raw['is_train']=1
test_raw['is_train']=0

df = pd.concat([train_raw, test_raw],sort=False)

rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

# 3.数据清洗

In [3]:
# #原始路线
# all_data = df.copy()
# bad_feature = ['env_t', 'V_A', 'V_B', 'V_C', 'I_B', 'I_C', 'efficiency', 'efficiency_A', 'efficiency_B', 'efficiency_C']
# bad_index1 = all_data[bad_feature][
#     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
#     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# ].dropna(how='all').index
# bad_index2 = all_data[
#     ((all_data['V_A']<500)&(all_data['V_A']!=0))|
#     ((all_data['V_B']<500)&(all_data['V_B']!=0))|
#     ((all_data['V_C']<500)&(all_data['V_C']!=0))].index
# bad_index = pd.Int64Index(list(bad_index1)+list(bad_index2))

# # bad_index = all_data[bad_feature][
# #     (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
# #     (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
# # ].dropna(how='all').index



# bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)






# # 上下记录均值替代异常值
# for idx, line in bad_data.iterrows():
#     ID = line['ID']
#     col_index = line[bad_feature][ 
#         (line[bad_feature] > all_data[bad_feature].mean() + 3 * all_data[bad_feature].std())| 
#         (line[bad_feature] < all_data[bad_feature].mean() - 3 * all_data[bad_feature].std())
#     ].index
#     index = all_data[all_data['ID'] == ID].index
    
#     # idx - before_offset, CV  0.8684
#     before_offset = 1
#     while (idx + before_offset)in bad_index:
#         before_offset += 1

#     after_offset = 1
#     while (idx + after_offset) in bad_index:
#         after_offset += 1
    
#     replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
#     all_data.loc[index, col_index] = replace_value[0]

    


In [4]:
rolling_mask_two = [-i for i in range(1,2)]+[i for i in range(1,2)]
rolling_mask_four = [-i for i in range(1,3)]+[i for i in range(1,3)]
rolling_mask_six = [-i for i in range(1,4)]+[i for i in range(1,4)]
rolling_mask_eight = [-i for i in range(1,5)]+[i for i in range(1,5)]
rolling_mask_ten = [-i for i in range(1,6)]+[i for i in range(1,6)]

In [5]:
#计算偏差率的辅助列
# for c in ['I_A','I_B','I_C','V_A','V_B','V_C']:
for c in ['I_A','I_B','I_C']:
    df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
    df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
    df[c+'_cor'] = df[c]
    
    #out of range
    oor_index = df[df[c]>20].index
    print(oor_index)
    
#     outlier_index = df[df[c+'_exception_ratio']>1.6].index
#     print(outlier_index)
    
#     ab_index = pd.Int64Index(set(list(oor_index)+list(outlier_index)))
    ab_index = pd.Int64Index(set(list(oor_index)))
    print(ab_index)
    
    ab_data = df.loc[ab_index].sort_values(by='ID', ascending=True)
    
    print(ab_data[['ID', 'V_A', 'V_B', 'V_C', 'I_A', 'I_B', 'I_C', 'y']])
    
    # 上下记录均值替代异常值
    for idx, line in ab_data.iterrows():
        ID = line['ID']
        value = line[c]
        
        index = df[df['ID'] == ID].index
            
        before_offset = 1
        while (idx - before_offset)in ab_index:
            before_offset += 1

        after_offset = 1
        while (idx + after_offset) in ab_index:
            after_offset += 1
    
        print('ID :' + str(ID))
        print(value, 'is abnormal as value of ',c)
        replace_value = (df.loc[index - before_offset, c].values + df.loc[index + after_offset, c].values) / 2
        df.loc[index, c+'_cor'] = replace_value[0]
        print('Has been replaced by '+str(replace_value[0]))
    
    df[c] = df[c+'_cor']
    df.drop(columns=[c+'_cor',c+'_exception_ratio',c+'_avg_sequence'],axis=1,inplace=True)
    

# Corrections for presumed outlier points for I_A failed.....  cv drops continuously    
    
# for c in ['I_A']:
#     df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
#     df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
#     df[c+'_cor'] = df[c]
    
# #     #out of range
# #     oor_index = df[df[c]>20].index
# #     print(oor_index)
    
#     outlier_index = df[df[c+'_exception_ratio']>4].index
#     print('outlier_index :\n',outlier_index)
    
# #     ab_index = pd.Int64Index(set(list(oor_index)+list(outlier_index)))
#     ab_index = pd.Int64Index(set(list(outlier_index)))
#     print(ab_index)
    
#     ab_data = df.loc[ab_index].sort_values(by='ID', ascending=True)
    
#     print(ab_data[['ID', 'V_A', 'V_B', 'V_C', 'I_A', 'I_B', 'I_C', 'y']])
    
#     # 上下记录均值替代异常值
#     for idx, line in ab_data.iterrows():
#         ID = line['ID']
#         value = line[c]
        
#         index = df[df['ID'] == ID].index
            
#         before_offset = 1
#         while (idx - before_offset)in ab_index:
#             before_offset += 1

#         after_offset = 1
#         while (idx + after_offset) in ab_index:
#             after_offset += 1
    
#         print('ID :' + str(ID))
#         print(value, 'is abnormal as value of ',c)
#         replace_value = (df.loc[index - before_offset, c].values + df.loc[index + after_offset, c].values) / 2
#         df.loc[index, c+'_cor'] = replace_value[0]
#         print('Has been replaced by '+str(replace_value[0]))
    
#     df[c] = df[c+'_cor']
#     df.drop(columns=[c+'_cor',c+'_exception_ratio',c+'_avg_sequence'],axis=1,inplace=True)

        
        
        
        
        
for c in ['V_A','V_B','V_C']:
    df[c+'_avg_sequence'] = np.nanmean([df[c].shift(i) for i in rolling_mask_eight],axis=0)
    df[c+'_exception_ratio'] = np.abs(df[c]-df[c+'_avg_sequence'])/df[c+'_avg_sequence']
    df[c+'_cor'] = df[c]
    
    #out of range
    oor_index = df[(df[c]>800)|((df[c]<500)&(df[c]!=0))].index
    print(oor_index)
    
#     outlier_index = df[df[c+'_exception_ratio']>1.6].index
#     print(outlier_index)
    
#     ab_index = pd.Int64Index(set(list(oor_index)+list(outlier_index)))
    ab_index = pd.Int64Index(set(list(oor_index)))
    print(ab_index)
    
    ab_data = df.loc[ab_index].sort_values(by='ID', ascending=True)
    
    print(ab_data[['ID', 'V_A', 'V_B', 'V_C', 'I_A', 'I_B', 'I_C', 'y']])
    
    # 上下记录均值替代异常值
    for idx, line in ab_data.iterrows():
        ID = line['ID']
        value = line[c]
        
        index = df[df['ID'] == ID].index
            
        before_offset = 1
        while (idx - before_offset)in ab_index:
            before_offset += 1

        after_offset = 1
        while (idx + after_offset) in ab_index:
            after_offset += 1
    
        print('ID :' + str(ID))
        print(value, 'is abnormal as value of ',c)
        replace_value = (df.loc[index - before_offset, c].values + df.loc[index + after_offset, c].values) / 2
        df.loc[index, c+'_cor'] = replace_value[0]
        print('Has been replaced by '+str(replace_value[0]))
    
    df[c] = df[c+'_cor']
    df.drop(columns=[c+'_cor',c+'_exception_ratio',c+'_avg_sequence'],axis=1,inplace=True)

        
        

        
df['P_A']=df['I_A']*df['V_A']
df['P_B']=df['I_B']*df['V_B']
df['P_C']=df['I_C']*df['V_C']
df['P_avg']=1/3*(df['P_A']+df['P_B']+df['P_C'])


# df.drop(columns=['I_A','I_B','I_C','V_A','V_B','V_C','P_A','P_B','P_C','P_avg'],axis=1,inplace=True)
# df.drop(columns=['I_A_avg_sequence','I_A_exception_ratio','I_B_avg_sequence','I_B_exception_ratio','I_C_avg_sequence','I_C_exception_ratio','V_B_avg_sequence','V_B_exception_ratio','V_A_avg_sequence','V_A_exception_ratio','V_C_avg_sequence','V_C_exception_ratio',],axis=1,inplace=True)


Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Empty DataFrame
Columns: [ID, V_A, V_B, V_C, I_A, I_B, I_C, y]
Index: []
Int64Index([14, 859, 981, 1065, 1237, 1419, 3303, 3305, 3309, 3311, 3313], dtype='int64')
Int64Index([3303, 1065, 3305, 1419, 3309, 14, 3311, 3313, 1237, 981, 859], dtype='int64')
        ID    V_A  V_B  V_C   I_A     I_B   I_C         y
14      22  65382    7  107  7.19  645.39  2.75  5.440741
859    948  65394    4   14  6.87  638.94  2.37       NaN
981   1070  65477   41  692  2.62  653.71  3.15       NaN
1065  1173  65408   22  250  7.04  649.75  2.75  7.753474
1237  1362  65386   23  244  7.00  652.02  2.79  7.806384
1419  1565  65420    8  260  6.93  653.23  0.83       NaN
3303  3597  65419   10   63  6.93  643.45  2.70       NaN
3305  3599  65419    5   64  6.97  643.58  3.35  0.268685
3309  3603  65420    3   67  7.01  643.75  3.16  0.286363
3311  3605  65420   11   67  7.01  643.83  3.36  0.278587
3313  3607  65420   15   68  7.09  643.94  0.03  

Int64Index([   14,   499,   527,   528,   529,   530,   531,   673,   859,
              981,  1065,  1066,  1067,  1178,  1237,  1326,  1373,  1419,
             1520,  1561,  1722,  1950,  2036,  2071,  2214,  2364,  2454,
             2567,  2629,  2670,  2890,  3108,  3136,  3253,  3254,  3255,
             3303,  3305,  3309,  3311,  3313,  3316,  3317,  3318,  3429,
             4068,  5083,  5488,  5565,  6976,  7281, 14216, 15833, 15971],
           dtype='int64')
Int64Index([ 2567, 14216,  1419,    14,   527,   528,   529,   530,   531,
             2454,  2071,  1561,  1178,  1950,   673,  3108,  2214,  1065,
             1066,  1067,  1326,  3253,  3254,  3255,  1722,  2364,  5565,
             3136,  6976,  2629,  2890,   981,  1237, 15833,   859,  5083,
             1373, 15971,  4068,  3429,  3303,  3305,  3309,  2670,  3311,
             1520,  3313,  5488,   499,  2036,  3316,  3317,  3318,  7281],
           dtype='int64')
          ID    V_A    V_B    V_C   I_A    I_B

26.0 is abnormal as value of  V_C
Has been replaced by 685.5
ID :2797.0
17.0 is abnormal as value of  V_C
Has been replaced by 686.5
ID :2875.0
23.0 is abnormal as value of  V_C
Has been replaced by 683.0
ID :2916.0
30.0 is abnormal as value of  V_C
Has been replaced by 676.0
ID :2986.0
89.0 is abnormal as value of  V_C
Has been replaced by 686.5
ID :3149.0
5.0 is abnormal as value of  V_C
Has been replaced by 668.5
ID :3152.0
260.0 is abnormal as value of  V_C
Has been replaced by 669.5
ID :3393.0
65470.0 is abnormal as value of  V_C
Has been replaced by 655.0
ID :3421.0
65475.0 is abnormal as value of  V_C
Has been replaced by 701.5
ID :3538.0
65463.0 is abnormal as value of  V_C
Has been replaced by 658.0
ID :3539.0
65463.0 is abnormal as value of  V_C
Has been replaced by 658.0
ID :3540.0
18.0 is abnormal as value of  V_C
Has been replaced by 658.0
ID :3597.0
63.0 is abnormal as value of  V_C
Has been replaced by 698.0
ID :3598.0
63.0 is abnormal as value of  V_C
Has been replaced 

# 4.特征工程

## 4.1.增加前后有效发电量均值

In [6]:
# df = all_data.copy()



#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = df.shape[0]

i_y =df.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(df.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(df.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(df.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(df.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(df.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(df.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)
        

df['next_value'] = next_one
df['prev_value'] = prev_one
df['avg_value'] = np.nanmean([df['next_value'], df['prev_value']],axis=0)

df.drop(['next_value','prev_value'],1,inplace=True)



## 4.2.增加前后功率均值

In [7]:
def add_avg(df):
    array = np.array(df["P_avg"])
    newarray=[]
    num = 0
    for i in np.arange(len(array)):
        for j in np.arange(10):
            if i<10:
                num = (array[j-1]+array[j-2]+array[j-3])/3
            if i>=10:
                num = (array[i-1]+array[i-2]+array[i-3]+array[i-5]+array[i-6]+array[i-7]+array[i-8]+array[i-9])/9
        newarray.append(num)
    df["old_SoCalledSF_P_avg"] = newarray
    return df

df = add_avg(df)

# 5.训练集测试集数据准备

## 5.1.去除训练集的重复样本

In [8]:
# 拆分数据

train_data = df[df['is_train']==1]
test_data = df[df['is_train']==0]
len(train_data), len(test_data)

(9000, 8409)

In [31]:
# 准备提交结果

df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

In [10]:
# l_drop_base=['I_A','I_B','I_C','V_A','V_B','V_C','P_A','P_B','P_C','P_avg']
# l_drop =[]
# for c in l_drop_base:
#     l_drop.append(c+'_cor')
# print(l_drop)
    

In [11]:
train_data.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,P_A,P_B,P_C,P_avg,wind_speed,wind_direction,y,is_train,avg_value,old_SoCalledSF_P_avg
2,10,-19.14,-17.4,34,80.55,106.32,16.98,118.36,729.0,709.0,...,976.86,155.98,1087.50,740.113333,0.6,272,1.437752,1,1.692575,1172.807778
3,11,-18.73,-17.3,30,99.90,139.00,21.20,139.51,728.0,717.0,...,1128.40,172.08,1132.56,811.013333,0.8,275,1.692575,1,1.706770,1172.807778
4,12,-17.54,-17.0,41,82.48,114.86,14.91,117.66,731.0,722.0,...,1279.25,166.06,1310.40,918.570000,1.1,283,1.975787,1,2.031615,1172.807778
6,14,-15.43,-16.6,53,73.98,101.72,15.55,104.67,730.0,727.0,...,1474.60,225.37,1517.34,1072.436667,0.9,280,2.370656,1,2.253939,1172.807778
7,15,-14.60,-16.3,65,64.62,86.86,13.09,93.92,727.0,729.0,...,1548.51,233.28,1674.40,1152.063333,1.1,280,2.532091,1,2.575187,1172.807778


In [12]:
# print('去重前训练集条数:' +str(train_data.shape[0]))
# # train_data = train_data.drop_duplicates(train_data.columns.drop(['ID','avg_value','old_SoCalledSF_P_avg']), keep='first')
# # train_data = train_data.drop_duplicates(train_data.columns.drop(['ID','avg_value','old_SoCalledSF_P_avg']+l_drop), keep='first')
# train_data = train_data.drop_duplicates(train_data.columns.drop(['ID','avg_value','old_SoCalledSF_P_avg']), keep='first')
# print('去重后训练集条数:' +str(train_data.shape[0]))

去重前训练集条数:9000
去重后训练集条数:8918


## 5.2.使训练集样本分布更合理

In [13]:
# def improve_train_test_data(train_data, test_data, poly=False, select=False):
#     Y = train_data['y']
#     X = train_data.drop(['y','ID','is_train'], axis=1)
#     test_data = test_data.drop(['y','ID','is_train'], axis=1)
    
#     polynm = None
#     if poly:
#         from sklearn.preprocessing import PolynomialFeatures
#         polynm = PolynomialFeatures(degree=2, interaction_only=True)
#         X = polynm.fit_transform(X)
#         test_data = polynm.transform(test_data)
        
#     X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=123)
    
#     sm = None
#     if select:
#         from sklearn.feature_selection import SelectFromModel
#         sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
#         X_train = sm.fit_transform(X_train, Y_train)
#         X_val = sm.transform(X_val)
#         test_data = sm.transform(test_data)
        
#     train_X = np.concatenate([X_train, X_val])
#     train_Y = np.concatenate([Y_train, Y_val])

# #     sm = None
# #     if select:
# #         from sklearn.feature_selection import SelectFromModel
# #         sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
# #         X = sm.fit_transform(X, Y)
# #         test_data = sm.transform(test_data)
    
# #     train_X = X
# #     train_Y = Y
#     test_X = test_data
        
#     return train_X, train_Y, test_X

In [14]:
def improve_train_test_data(train_data, test_data, poly=False, select=False):
    Y = train_data['y']
    X = train_data.drop(['y','is_train','I_B','I_C'], axis=1)
    test_X = test_data.drop(['y','is_train','I_B','I_C'], axis=1)
    
    return X, Y, test_X

In [15]:
# # X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data, poly=True, select=True)
# X_train, X_test, y_train, y_test, sub_data, sm, polynm = generate_train_data(train_data, test_data)

# train_X = np.concatenate([X_train, X_test])
# train_Y = np.concatenate([y_train, y_test])



# test_X = sub_data

In [16]:
# # train_X, train_Y, test_X = improve_train_test_data(train_data, test_data, poly=True, select=True)
train_X, train_Y, test_X = improve_train_test_data(train_data, test_data, poly=False, select=False)

# 6.Cross Validation

In [17]:
# 定义打分函数,  SCORE = 1/(1+RMSE)
def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))

# def cal_score(mse):
#     return np.divide(1, 1 + np.sqrt(mse))

# 定义交叉验证函数  
def cross_validate(models, X, Y, cv=5):
    model_name, mse_avg, score_avg, rmse_r, score_r = [], [], [], [], []
    for i, model in enumerate(models):
        #获取模型名
        name = str(i + 1) + '.' + str(model) 
#         print(i + 1,'- Model:', str(model).split('(')[0])
        print(name)
#         model_name.append(str(i + 1) + '.' + str(model).split('(')[0])
        model_name.append(name.split('(')[0])
        #计算metric
        
        skf = KFold(5, shuffle=True, random_state=1)
        Y_pred = np.zeros(X.shape[0])
        for train_idx,val_idx in skf.split(X,Y):
            print('train_idx: \n',str(len(train_idx)),train_idx)
            print('val_idx: \n',str(len(val_idx)),val_idx)
            X_train, X_val, Y_train, Y_val = X.iloc[train_idx], X.iloc[val_idx], Y.iloc[train_idx], Y.iloc[val_idx]
            model.fit(X_train, Y_train)
            Y_pred[val_idx] = model.predict(X_val)
            print('Y_pred: \n',Y_pred)
        rmse_c = np.sqrt(mean_squared_error(Y, Y_pred))
        score_c = cal_score(rmse_c)
        rmse_r.append(rmse_c)
        score_r.append(score_c)
        
        
#         folds=strat_k_fold.split(X,Y)
        #apply shuffling to cross_val_score
#         strat_k_fold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=0)
        nmse = cross_val_score(model, X, Y, cv=cv, scoring='neg_mean_squared_error')
    
#         nmse = cross_val_score(model, X, Y, cv=cv, scoring='neg_mean_squared_error')
        avg_mse = np.average(-nmse)
        mse_avg.append(avg_mse)
        #计算分数
        scores = cal_score(-nmse)
        avg_score = np.average(scores)    
        score_avg.append(avg_score)
        print('MSE:', -nmse)
        print('Score:', scores)
        print('Average MSE:', avg_mse, ' - Score:', avg_score, ' - Score_c:', score_c, '\n')
    res = pd.DataFrame()
    res['Model'] = model_name
    res['Avg MSE'] = mse_avg
    res['Avg Score'] = score_avg
    res['Score Customize'] = score_r
    res['RMSE Customize'] = rmse_r
    return res

In [18]:
#基学习器

xgbt1 = xgb.XGBRegressor(n_estimators=950, max_depth=3, max_features='sqrt', random_state=321, n_jobs=8)
xgbt2 = xgb.XGBRegressor(n_estimators=1000, max_depth=3, max_features='sqrt', random_state=456, n_jobs=8)
xgbt3 = xgb.XGBRegressor(n_estimators=1100, max_depth=3, max_features='sqrt', random_state=789, n_jobs=8)
# n_estimators=1000  max_depth=5  'sqrt'  GradientBoostingRegressor 最佳参数 ,learning_rate=0.08
gbdt1 = GradientBoostingRegressor(n_estimators=800, max_depth=4, max_features='log2', random_state=123,learning_rate=0.08)
gbdt2 = GradientBoostingRegressor(n_estimators=900, max_depth=4, max_features='log2', random_state=456,learning_rate=0.08)
gbdt3 = GradientBoostingRegressor(n_estimators=1000, max_depth=5, max_features='log2', random_state=789,learning_rate=0.08)
# n_estimators=700, max_features='auto', random_state=2, n_jobs=8,max_depth=10
forest1 = RandomForestRegressor(n_estimators=800, max_features='sqrt', random_state=7, n_jobs=8)
forest2 = RandomForestRegressor(n_estimators=900, max_features='log2', random_state=9, n_jobs=8)
forest3 = RandomForestRegressor(n_estimators=900, max_features='sqrt', random_state=11, n_jobs=8) 

lgb1 = LGBMRegressor(n_estimators=900, max_depth=5, random_state=5, n_jobs=8) 
lgb2 = LGBMRegressor(n_estimators=850, max_depth=4, random_state=7, n_jobs=8)
lgb3 = LGBMRegressor(n_estimators=720, max_depth=4, random_state=9, n_jobs=8)
# lgb3 = LGBMRegressor(n_estimators=1000, max_depth=5, random_state=9, )

In [19]:
train_data.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,P_A,P_B,P_C,P_avg,wind_speed,wind_direction,y,is_train,avg_value,old_SoCalledSF_P_avg
2,10,-19.14,-17.4,34,80.55,106.32,16.98,118.36,729.0,709.0,...,976.86,155.98,1087.50,740.113333,0.6,272,1.437752,1,1.692575,1172.807778
3,11,-18.73,-17.3,30,99.90,139.00,21.20,139.51,728.0,717.0,...,1128.40,172.08,1132.56,811.013333,0.8,275,1.692575,1,1.706770,1172.807778
4,12,-17.54,-17.0,41,82.48,114.86,14.91,117.66,731.0,722.0,...,1279.25,166.06,1310.40,918.570000,1.1,283,1.975787,1,2.031615,1172.807778
6,14,-15.43,-16.6,53,73.98,101.72,15.55,104.67,730.0,727.0,...,1474.60,225.37,1517.34,1072.436667,0.9,280,2.370656,1,2.253939,1172.807778
7,15,-14.60,-16.3,65,64.62,86.86,13.09,93.92,727.0,729.0,...,1548.51,233.28,1674.40,1152.063333,1.1,280,2.532091,1,2.575187,1172.807778


In [20]:
# 
import lightgbm

def my_val(preds, train_data):
    label = train_data.get_label()
    return 'score', 1/(1+np.sqrt(mean_squared_error(preds, label))), True
def my_obj(preds, train_data):
    labels = train_deata.get_label()

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.08,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 0
}
# 同样的条件下，此参数设置达到local cv: 0.8575832021441638

train = train_data.copy()
test = test_X.copy()


test_predicts = []
val_preds = []

# log_test_predicts = []
# log_val_predicts = []
for idx, seed in enumerate([1,2,3,4,5]):
# for idx, seed in enumerate([1]):
    kf = KFold(5, shuffle=True, random_state=seed)
    
    val_preds.append(np.zeros(train.shape[0]))
    for n_fold, (tra_idx, val_idx) in enumerate(kf.split(train)):
        tra = train.iloc[tra_idx]
        val = train.iloc[val_idx]
        tst = test.copy()

#         grouper = ['bin_ls','bin_env_t','bin_board_t']
#         x = tra.groupby(grouper)['y'].agg({'target_encoding_mean':'mean',
#                                            'target_encoding_max':'max',
#                                            'target_encoding_min':'min',
#                                            'target_encoding_median':'median',
#                                            'target_encoding_var':'var',
#                                            'target_encoding_count':'count'}).reset_index()
#         tra = tra.merge(x, on=grouper, how='left')
#         val = val.merge(x, on= grouper , how ='left')
#         tst = tst.merge(x, on=grouper, how='left')
        predictor = [c for c in tra.columns.tolist() if c not in['y','I_B','I_C','is_train']]

        train_set = lightgbm.Dataset(
            tra[predictor],
            tra['y']
        )

        validation_set = lightgbm.Dataset(
            val[predictor],
            val['y']
        )

        model = lightgbm.train(params, train_set, num_boost_round=5000,
                              valid_sets= [validation_set],
                              valid_names=['valid'],
                              early_stopping_rounds=100,
                               feval=my_val,
                              verbose_eval=500)

        val_preds[idx][val_idx] = model.predict(val[predictor])
        test_predicts.append(model.predict(tst[predictor]))

print('local cv:',1/(1+np.sqrt(mean_squared_error(train['y'],np.mean(val_preds,axis=0)))))


Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0270214	valid's score: 0.858825
[1000]	valid's l2: 0.0263334	valid's score: 0.860381
[1500]	valid's l2: 0.0259996	valid's score: 0.861145
Early stopping, best iteration is:
[1756]	valid's l2: 0.0259051	valid's score: 0.861363
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[363]	valid's l2: 0.017142	valid's score: 0.88423
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0581497	valid's score: 0.805709
Early stopping, best iteration is:
[811]	valid's l2: 0.0575354	valid's score: 0.806539
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0259014	valid's score: 0.861372
[1000]	valid's l2: 0.0252068	valid's score: 0.862987
Early stopping, best iteration is:
[1077]	valid's l2: 0.0250968	valid's score: 0.863245
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.01

In [21]:
regrs = [
    xgbt1, gbdt1, forest1, lgb1,
    xgbt2, gbdt2, forest2, lgb2,
    xgbt3, gbdt3, forest3, lgb3
]

regrs_light = [
    lgb3, xgbt3, gbdt3, forest3
]

In [22]:
# cross_validate(models=regrs_light, X = train_X, Y = train_Y)

In [23]:
cross_validate(models=regrs_light, X = train_X, Y = train_Y)

1.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=720,
       n_jobs=8, num_leaves=31, objective=None, random_state=9,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
train_idx: 
 7134 [   0    1    2 ... 8915 8916 8917]
val_idx: 
 1784 [   6    7   16 ... 8899 8903 8912]
Y_pred: 
 [0. 0. 0. ... 0. 0. 0.]
train_idx: 
 7134 [   0    1    2 ... 8914 8915 8917]
val_idx: 
 1784 [   3    4    5 ... 8907 8911 8916]
Y_pred: 
 [0.         0.         0.         ... 0.         9.54075576 0.        ]
train_idx: 
 7134 [   0    2    3 ... 8912 8913 8916]
val_idx: 
 1784 [   1   12   13 ... 8914 8915 8917]
Y_pred: 
 [ 0.          1.69522452  0.         ... 10.38895588  9.54075576
  8.97770824]
train_idx: 
 7135 [   0    1    2 ... 8915 8916 8917]
val_idx: 
 1783 [   8    9   10 .

KeyboardInterrupt: 

# 7.Stack

In [ ]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # Train_X: 原始训练集输入矩阵, Train_Y: 原始训练集输出矩阵, Test_X: 原始测试集输入矩阵
    def fit_predict(self, Train_X, Train_Y, Test_X):
        Train_X = np.array(Train_X)
        Train_Y = np.array(Train_Y)
        Test_X = np.array(Test_X)

        folds = list(KFold(n_splits=self.n_splits, shuffle=True, random_state=2018).split(Train_X, Train_Y))       
        
        # 以基学习器预测结果为特征的 stacker训练数据 与 stacker预测数据
        # 原始训练集预测结果容器
        S_train = np.zeros((Train_X.shape[0], len(self.base_models)))
        # 原始测试集预测结果容器
        S_predict = np.zeros((Test_X.shape[0], len(self.base_models)))
        
        for n_model, regr in enumerate(self.base_models):
            print(n_model + 1, 'Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((Test_X.shape[0], self.n_splits))
            
            for n_fold, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train_fold, Y_train_fold, X_test_fold, Y_test_fold = Train_X[train_idx], Train_Y[train_idx], Train_X[test_idx], Train_Y[test_idx]
                print ('Fit fold', (n_fold+1), '...')
                regr.fit(X_train_fold, Y_train_fold)
                Y_pred = regr.predict(X_test_fold)
                # 每折训练得到的模型根据原始训练集中的测试折的输入矩阵预测
                S_train[test_idx, n_model] = Y_pred
                # 每折训练得到的模型根据原始测试集输入矩阵预测
                S_predict_i[:, n_fold] = regr.predict(Test_X)
            
            S_predict[:, n_model] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, Train_Y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, Train_Y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [ ]:
stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs_light)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

In [ ]:
# Stack with 12 models

stacking_model = SVR(C=100, gamma=0.01, epsilon=0.01)
stacker = Stacker(5, stacking_model, regrs)
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(train_X, train_Y, test_X)

In [ ]:
# df_result['score'] = pred_stack

# index = df_result[df_result['ID'].isin(special_missing_ID)].index
# df_result.loc[index, 'score'] = 0.379993053

# df_result.to_csv('../result/081703_08785.csv', index=False, header=False)

In [36]:
best = pd.read_csv('../result/080203_086375_tosubmit.csv')

In [37]:
best.shape

(8409, 2)

In [38]:
special_missing_ID.shape

(46,)

In [39]:
index = best[best['ID'].isin(special_missing_ID)].index
best.loc[index, 'y'] = 0.379993053

best.to_csv('../result/081901_086375.csv', index=False, header=False)

In [40]:
best.head()

,ID,y
0,1,0.379993
1,9,1.403857
2,13,2.149638
3,17,3.374738
4,18,3.618319
